In [1]:
from Generate import Generate
from Traversal_Glove import Meta_Poetry_Glove

from nltk.corpus import wordnet as wn
from gensim.models import KeyedVectors

from functions import *

In [2]:
glove = KeyedVectors.load_word2vec_format("/Users/chris/Downloads/glove.6B/glove.6B.300d.w2v.txt")

In [3]:
g = Generate(wv=glove)

In [4]:
mp = Meta_Poetry_Glove(wv=glove)

In [46]:
seed = wn.synset('straw.n.01')

In [47]:
mp.print_five_words_glove_specify(seed)

wear->hats->straw~~>scattering->particles


In [48]:
words = mp.five_word_algorithm_glove_specify(seed)

In [50]:
g.get_postag_dict()[2]['hats']

[]

In [25]:
%%time

lines = []
for i in range(1, len(words)):
    w1 = words[i - 1][0]
    w2 = words[i][0]
    template, k = g.generate_line(w1, w2)
    lines.append((k[0][1][1], k[0][0]))

['aircraft', 'air']


IndexError: list index out of range

In [18]:
# IndexError with easily and chicken (still commas)

In [22]:
g.generate_line('food', 'chicken', template=['DT', 'NN', 'VBN', 'IN', 'DT', 'NN', ','])

['food', 'chicken']
['DT', 'NN', 'VBN', 'IN', 'DT', 'NN', ',']
First noun: chicken
Last noun: food
['NN', 'VBN', 'IN', 'DT']
#################################

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
Generation took 3.548 seconds
6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model/model.ckpt-227699


(['DT', 'NN', 'VBN', 'IN', 'DT', 'NN', ','], [])

In [23]:
g.place_words_in_template(['food', 'chicken'], ['DT', 'NN', 'VBN', 'IN', 'DT', 'NN', ','])

['food', 'chicken']


[['DT', 'NN', 'VBN', 'IN', 'DT', 'NN', ','], [1, 5], ['chicken', 'food']]

In [25]:
g.genPoem_forward('chicken', ['NN', 'VBN', 'IN', 'DT'])

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
Generation took 3.417 seconds


[]

In [9]:
saved_args

Namespace(batch_size=1, data_dir='data/frost', decay_rate=0.9, dropout=0.3, embedding_dim=300, gpu_mem=0.99, grad_clip=5.0, init_from='save_3', input_encoding=None, learning_rate=0.0002, log_dir='logs', model='lstm', num_epochs=100, num_layers=3, reverse=0, rnn_size=1000, save_dir='save_2', save_every=10000, seq_length=1, test_split=0.0, trainable_embeddings=0, vocab_size=6885)

In [22]:
from functions import *
import os
from six.moves import cPickle
import tensorflow as tf
from model_forw import Model as Model_forw

tf.reset_default_graph()
with open(os.path.join(g.get_save_dir(), 'config.pkl'), 'rb') as f:
    saved_args = cPickle.load(f)
with open(os.path.join(g.get_save_dir(), 'words_vocab.pkl'), 'rb') as f:
    word_keys, vocab = cPickle.load(f)
model = Model_forw(saved_args, True)
print (len(vocab))
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    saver = tf.train.Saver(tf.global_variables())
    ckpt = tf.train.get_checkpoint_state(g.get_save_dir())
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
        poem = []
        line_num = 0
        wordPool_ind = 0
        state = sess.run(model.initial_state)
        init_score = np.array([[0]])
        lst = search_forward(model, vocab, init_score,['chicken'],state, sess, 1,\
                          g.get_dict_tag(),g.get_dict_pos(),g.get_width(),g.get_wordPools()[wordPool_ind], 
                             g.get_pos_set(), ['NN', 'VBN', 'IN', 'DT'])
        print(lst)
        next_search = (init_score, ((init_score, state), ['chicken']))
        beamSearchOneLevel(model, vocab, next_search[1][0][0], next_search[1][1],\
                                next_search[1][0][1], sess, 1,\
                                g.get_dict_tag(), g.get_dict_pos(), g.get_width(), g.get_wordPools()[wordPool_ind],
                                   g.get_pos_set(), ['NN', 'VBN', 'IN', 'DT'])


6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
[]


KeyError: 'chicken'

In [18]:
def beamSearchOneLevel(model, vocab, prob_sequence, sequence, state, session, \
                    temp, dictPartSpeechTags,dictPossiblePartsSpeech, breadth, wordPool, PartOfSpeachSet, TemplatePOS):
        def decayRepeat(word,sequence, scale):
            safe_repeat_words = []
            #safe_repeat_words = set(["with,the,of,in,i"])
            score_adjust = 0
            decr = -scale
            for w in range(len(sequence)):
                if(word==sequence[w] and word not in safe_repeat_words):
                    score_adjust += decr
                decr += scale/10 #decreases penalty as the words keep getting further from the new word
            return score_adjust
        def partsOfSpeechFilter(word1,word2,dictPartSpeechTags,dictPossiblePartsSpeech):
            okay_tags = set(["RB","RBR","RBS"]) #THESE ARE THE ADVERBS
            try:
                tag1 = dictPartSpeechTags[word1]
            except KeyError:
                return True
            tag2 = dictPartSpeechTags[word2]
            #if(tag1==tag2 and tag1 not in okay_tags):
            #    return True
            if(tag1 not in dictPossiblePartsSpeech[tag2]):
                return True
            else:
                return False
        if(len(sequence)==len(TemplatePOS)):
            return("begin_line")
        ret = []
        scale = .02 #scale is the significant magnitude required to affect the score of bad/good things
        dist, state = model.compute_fx(session, vocab, prob_sequence, sequence, state, temp)
        #for pred_stress in list(fsaLine[post_stress].prevs):
        word_set = set([])
        #print (len(sequence))
        for word in list(set(PartOfSpeachSet[TemplatePOS[-len(sequence)-1]])):
            #PREVENTS REPEAT ADJACENT WORDS OR PROBLEM-TAGGED WORDS
            if(word == sequence[0]):
                continue
            if(partsOfSpeechFilter(word,sequence[0],dictPartSpeechTags,dictPossiblePartsSpeech)):
                continue
            #FACTORS IN SCORE ADJUSTMENTS
            score_adjust = decayRepeat(word, sequence, 100*scale) #repeats
            score_adjust += scale*len(word)/50 #length word
            if(word in wordPool):
                score_adjust += scale
            #CALCULATES ACTUAL SCORE
            key = np.array([[vocab[word]]])
            new_prob = dist[key]
            score_tuple = (new_prob, state)
            score_tup = (score_tuple[0]+score_adjust,score_tuple[1]) #NOTE SHOULD SCORE_ADJUST BE ADDED HERE OR JUST IN THE ITEM LINE?
            item = (score_tup[0],(score_tup, [word]+sequence))
            if(item[0]==[[-float("inf")]]):
                continue
            ret+=[item]
        return ret

# Experiments

In [5]:
import tensorflow as tf
from model_forw import Model as Model_forw
from model_back import Model as Model_back
import os
from six.moves import cPickle

In [6]:
def force_middle(model, vocab, prob_sequence, sequence, sess, lst, next_word):
    def decayRepeat(word,sequence, scale):
        safe_repeat_words = []
        #safe_repeat_words = set(["with,the,of,in,i"])
        score_adjust = 0
        decr = -scale
        for w in range(len(sequence)):
            if(word==sequence[w] and word not in safe_repeat_words):
                score_adjust += decr
            decr += scale/10 #decreases penalty as the words keep getting further from the new word
        return score_adjust

    poem = []
    line_num = 0
    wordPool_ind = 0
    ret=[]
    for tup in lst:
        state_init = tup[1][0][1]
        prob_seq =  tup[1][0][0]
        sequence=tup[1][1]
        scale = .02 #scale is the significant magnitude required to affect the score of bad/good things
        dist, state = model.compute_fx(sess, vocab, prob_seq, sequence, state_init, 1)
        #PREVENTS REPEAT ADJACENT WORDS OR PROBLEM-TAGGED WORDS
        if(next_word == sequence[-1]):
            continue
        #if(partsOfSpeechFilter(sequence[-1],next_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
        #    continue
        #FACTORS IN SCORE ADJUSTMENTS
        score_adjust = decayRepeat(next_word, sequence, 100*scale) #repeats
        score_adjust += scale*len(next_word)/50 #length word
        #if(next_word in wordPool):
        #    score_adjust += scale
        #CALCULATES ACTUAL SCORE
        key = np.array([[vocab[next_word]]])
        new_prob = dist[key]
        score_tuple = (new_prob, state)
        score_tup = (score_tuple[0]+score_adjust,score_tuple[1]) #NOTE SHOULD SCORE_ADJUST BE ADDED HERE OR JUST IN THE ITEM LINE?
        item = (score_tup[0],(score_tup, sequence+[next_word]))
        if(item[0]==[[-float("inf")]]):
            continue
        ret+=[item]
    ret.sort(key=itemgetter(0), reverse = True)
    return ret[:g.get_width()]


In [23]:
def generate_line_collocations(word1, word2, collocations):
    # check for collocations and fill in
    postag= g.pos_synset([word1, word2], g.get_postag_dict()[0])

    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])

    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    first_pos=postag[1][first_position]
    last_pos=postag[1][last_position]
    template=postag[0]

    line = ['' for i in range(len(template))]
    line[min(postag[1])] = first_noun
    line[max(postag[1])] = last_noun
    
    line = g.insert_collocations(template, line, collocations)
    
    # finding indices with words in them
    not_blank = []
    pairs = []
    for i in range(len(line)):
        if line[i] is not '':
            pairs.append(i)
        if len(pairs) > 1:
            not_blank.append((pairs[0], pairs[1]))
            pairs.pop(0)
    # if haven't made it to the end of the sentence
    if pairs[0] != len(line) - 1:
        not_blank.append((pairs[0], len(line) - 1))

    # iterating over pairs of filled in words, filling in between
    candidates = []

    first_time = True
    first_noun = line[not_blank[0][0]]
    first_pos = not_blank[0][0]
    for i, j in not_blank:
        last_noun = line[j]
        last_pos = j
        
        ### Matias's code
        
        if first_time:
            list_1= g.genPoem_forward(first_noun, template[first_pos:last_pos])
            list_2= g.force_middle(list_1[:50], last_noun)
            candidates.extend(list_2)
            first_time = False
            continue

        tf.reset_default_graph()
        with open(os.path.join(g.get_save_dir(), 'config.pkl'), 'rb') as f:
            saved_args = cPickle.load(f)
        with open(os.path.join(g.get_save_dir(), 'words_vocab.pkl'), 'rb') as f:
            word_keys, vocab = cPickle.load(f)
        model = Model_forw(saved_args, True)
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(g.get_save_dir())
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                poem = []
                line_num = 0
                wordPool_ind = 0
                tt_3=[]
                for element in candidates:
                    seq=element[1][1]
                    state = element[1][0][1]
                    init_score = element[1][0][0]
                    seq=element[1][1]
                    lst = search_forward(model, vocab, init_score,seq,state, sess, 1,\
                                      g.get_dict_tag(),g.get_dict_pos(),g.get_width(),
                                         g.get_wordPools()[wordPool_ind], g.get_pos_set(), template[first_pos:last_pos])
                    # case where are generating between
                    # two words and need to force last one
                    if last_noun is not '':
                        lst = force_middle(model, vocab, init_score, seq, sess, lst[:50], last_noun)
                    tt_3+=lst
                tt_3.sort(key=itemgetter(0), reverse = True)
                candidates = tt_3[:50]
    pre_backwards = candidates
        
    # Filling in reverse
    tf.reset_default_graph()
    with open(os.path.join(g.get_save_dir_back(), 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(g.get_save_dir_back(), 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_back(saved_args, True)
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state(g.get_save_dir_back())
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
            poem = []
            line_num = 0
            wordPool_ind = 0
            tt_4=[]
            for tup in candidates[:30]:
                seq=tup[1][1]
                init_score, state=model.score_a_list(sess, vocab, seq)
                lst = search_back(model, vocab, init_score,seq,state, sess, 1,\
                              g.get_dict_tag(), g.get_dict_pos(),g.get_width(),
                                  g.get_wordPools()[wordPool_ind], g.get_pos_set(), template)

                tt_4+=lst
        else:
            print(ckpt)
    tt_4.sort(key=itemgetter(0), reverse = True)
    candidates = tt_4

    return postag, not_blank, line, candidates, pre_backwards

In [18]:
# %%time

# poets = {'Frost':'Grammar/Robert_Frost.txt'}
# filters = [("NN", "NN"), ("JJ", "NN"), ("JJ", "JJ", "NN"), ("JJ", "NN", "NN"), ("NN", "JJ", "NN"), ("NN", "NN", "NN"), ("NN", "PRP", "NN")]
# colls_dict = build_get_collocation_dict(poets, filters=filters)

# with open('colls_dict.p', 'wb') as f:
#     cPickle.dump(colls_dict, f)

In [8]:
with open('colls_dict.p', 'rb') as f:
    colls_dict = cPickle.load(f)

In [24]:
line = generate_line_collocations('time', 'sound', colls_dict)

['time', 'sound']
['NN', 'NN']
6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
[(array([[-10.116582]], dtype=float32), ((array([[-10.116582]], dtype=float32), (LSTMStateTuple(c=array([[-1.57089189e-01,  4.68482345e-01, -5.13061225e-01,
        -2.59089559e-01,  3.95553000e-02, -4.33747768e-01,
         6.80315346e-02, -3.61653239e-01,  5.89990914e-01,
        -6.20989859e-01,  4.59161311e-01,  3.72432917e-01,
         3.65641147e-01, -8.32064599e-02,  6.05716169e-01,
        -4.93030436e-02,  1.74741745e-01,  1.00436814e-01,
        -4.52590317e-01, -1.83590531e-01, -4.73806322e-01,
         5.11925876e-01,  2.03217760e-01, -5.26223332e-03,
        -2.25294873e-01,  1.26899883e-01,  5.31966448e-01,
         4.25994158e-01,  4.31043684e-01,  7.06217706e-01,
         2.42179744e-02, -5.53595006e-01,  4.43281710e-01,
         4.34996605e-01, -1.86580420e-01, -3.93095523e-01,
        -6.96935475e-01, -3.97394709e-02,  1.72314942e-01,
        -7.40372762e-02, -1.77637

6885
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2/model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model/model.ckpt-227699


In [27]:
line[1]

[(1, 3), (3, 5)]

In [28]:
line[2]

['', 'suppose', '', 'time', '', 'sound']

In [29]:
line[0]

[['CC', 'VB', 'DT', 'NN', 'IN', 'NN'], [3, 5], ['time', 'sound']]

In [46]:
for i in range(len(line[3])):
    print(' {:0.3}: '.format(line[3][i][0][0][0]), end='')
    print(' '.join(line[3][i][1][1]))
    print('')

 -27.8: be suppose some time of sound

 -30.1: know suppose some time of sound

 -31.3: do suppose some time of sound

 -31.8: let suppose some time of sound

 -31.9: be suppose no time of sound

 -32.4: get suppose some time of sound

 -32.5: have suppose some time of sound

 -32.6: be suppose this time in sound

 -32.8: say suppose some time of sound

 -33.0: be suppose any time of sound

 -33.1: keep suppose some time of sound

 -33.1: get suppose no time of sound

 -33.1: be suppose the time of sound

 -33.2: find suppose some time of sound

 -33.2: know suppose no time of sound

 -33.3: know suppose any time of sound

 -33.3: be suppose all time for sound

 -33.5: let suppose any time of sound

 -33.7: do suppose all time for sound

 -33.8: let suppose no time of sound

 -33.9: see suppose some time of sound

 -33.9: know suppose all time for sound

 -34.0: know suppose any time in sound

 -34.2: be suppose no time in sound

 -34.2: be suppose any time in sound

 -34.2: see suppos


 -54.8: make suppose another time of sound

 -54.8: ere suppose no time from sound

 -54.8: go suppose the time from sound

 -55.0: go suppose the time at sound

 -55.1: find suppose another time of sound

 -55.1: see suppose another time for sound

 -55.2: live suppose the time on sound

 -55.2: ripen suppose the time on sound

 -55.2: leave suppose a time with sound

 -55.3: take suppose the time from sound

 -55.3: give suppose a time in sound

 -55.3: keep suppose another time of sound

 -55.4: go suppose a time from sound

 -55.5: do suppose another time of sound

 -55.7: take suppose a time at sound

 -55.7: keep suppose a time at sound

 -55.7: run suppose a time with sound

 -55.8: c suppose a time by sound

 -55.8: remember suppose the time by sound

 -55.9: get suppose a time by sound

 -56.0: steady suppose a time in sound

 -56.0: ripen suppose a time with sound

 -56.0: get suppose a time at sound

 -56.1: make suppose a time by sound

 -56.1: c suppose a time in sound

 